<a href="https://colab.research.google.com/github/prince001996/Tensorflow-2.0-learning-/blob/master/Tensorflow_2_0_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [0]:
 vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [0]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


In [0]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


17465344/17464789 [==============================] - 2s 0us/step


In [0]:
# Lets look at one review
#train_data[0]
len(train_data[0]), len(train_data[1])

(218, 189)

In [0]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [0]:
train_data[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     1,   194,
        1153,   194,  8255,    78,   228,     5,     6,  1463,  4369,
        5012,   134,    26,     4,   715,     8,   118,  1634,    14,
         394,    20,    13,   119,   954,   189,   102,     5,   207,
         110,  3103,    21,    14,    69,   188,     8,    30,    23,
           7,     4,   249,   126,    93,     4,   114,     9,  2300,
        1523,     5,   647,     4,   116,     9,    35,  8163,     4,
         229,     9,   340,  1322,     4,   118,     9,     4,   130,
        4901,    19,

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 38s 61ms/step - loss: 0.4307 - acc: 0.8011 - val_loss: 0.3865 - val_acc: 0.8356
Epoch 2/10
625/625 [==============================] - 38s 60ms/step - loss: 0.2447 - acc: 0.9068 - val_loss: 0.3067 - val_acc: 0.8760
Epoch 3/10
625/625 [==============================] - 38s 60ms/step - loss: 0.1824 - acc: 0.9345 - val_loss: 0.3690 - val_acc: 0.8544
Epoch 4/10
625/625 [==============================] - 38s 60ms/step - loss: 0.1547 - acc: 0.9456 - val_loss: 0.3145 - val_acc: 0.8836
Epoch 5/10
625/625 [==============================] - 38s 61ms/step - loss: 0.1293 - acc: 0.9546 - val_loss: 0.2908 - val_acc: 0.8916
Epoch 6/10
625/625 [==============================] - 38s 61ms/step - loss: 0.1131 - acc: 0.9611 - val_loss: 0.3511 - val_acc: 0.8816
Epoch 7/10
625/625 [==============================] - 38s 60ms/step - loss: 0.0975 - acc: 0.9677 - val_loss: 0.3419 - val_acc: 0.8802
Epoch 8/10
625/625 [==============================] - 38s 61ms

In [0]:
results = model.evaluate(test_data, test_labels)
print(results)

In [0]:
word_index = imdb.get_word_index()

In [0]:
len(word_index), word_index

(88584,
 {'fawn': 34701,
  'tsukino': 52006,
  'nunnery': 52007,
  'sonja': 16816,
  'vani': 63951,
  'woods': 1408,
  'spiders': 16115,
  'hanging': 2345,
  'woody': 2289,
  'trawling': 52008,
  "hold's": 52009,
  'comically': 11307,
  'localized': 40830,
  'disobeying': 30568,
  "'royale": 52010,
  "harpo's": 40831,
  'canet': 52011,
  'aileen': 19313,
  'acurately': 52012,
  "diplomat's": 52013,
  'rickman': 25242,
  'arranged': 6746,
  'rumbustious': 52014,
  'familiarness': 52015,
  "spider'": 52016,
  'hahahah': 68804,
  "wood'": 52017,
  'transvestism': 40833,
  "hangin'": 34702,
  'bringing': 2338,
  'seamier': 40834,
  'wooded': 34703,
  'bravora': 52018,
  'grueling': 16817,
  'wooden': 1636,
  'wednesday': 16818,
  "'prix": 52019,
  'altagracia': 34704,
  'circuitry': 52020,
  'crotch': 11585,
  'busybody': 57766,
  "tart'n'tangy": 52021,
  'burgade': 14129,
  'thrace': 52023,
  "tom's": 11038,
  'snuggles': 52025,
  'francesco': 29114,
  'complainers': 52027,
  'templarios'

In [0]:
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]


In [0]:
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [0]:
# while we're at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [0]:
#making a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.9344935]
[0.27735958]


In [0]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
#to upload our own file
"""from google.colab import files
path_to_file = list(files.upload().keys())[0]"""

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [0]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [0]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [0]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [0]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 4.5842603e-03  1.3953068e-03  1.0012606e-03 ... -2.6670808e-03
    4.8370371e-03 -6.0607013e-03]
  [ 8.3607286e-03  2.3874948e-03  2.4988861e-03 ... -4.8990510e-03
    8.1510842e-03 -1.0426581e-02]
  [ 3.9634802e-03  3.7012859e-03  1.2129892e-03 ... -6.3499552e-04
    9.6160383e-04 -4.5581739e-03]
  ...
  [ 1.2419546e-02 -1.3349028e-03 -4.1525415e-03 ... -2.2950685e-03
   -5.9643164e-03  5.1625310e-03]
  [ 1.2785386e-02  3.9405227e-03 -2.6655709e-03 ... -1.2408644e-03
   -5.7659307e-03  2.6604915e-03]
  [ 1.0399178e-02  2.2081407e-03 -8.5334033e-03 ...  3.3421633e-03
   -5.1507792e-03  1.4777230e-03]]

 [[-5.0435262e-04 -2.1652088e-03  6.6280511e-04 ... -2.8579048e-04
   -3.9311056e-04  6.3278638e-03]
  [ 7.0744473e-04 -1.1088157e-02  2.4139243e-03 ... -7.8747142e-03
   -6.4899074e-04  4.3420838e-03]
  [ 4.8958743e-04 -6.6892132e-03 -1.0411381e-03 ... -8.1809834e-03
    5.6384364e-03  6.9516767e-03]
  ...
  [ 1.8430450e-03 -4.7759432e-04 -2.7263705e-03 ... -4.1753273e

In [0]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00458426  0.00139531  0.00100126 ... -0.00266708  0.00483704
  -0.0060607 ]
 [ 0.00836073  0.00238749  0.00249889 ... -0.00489905  0.00815108
  -0.01042658]
 [ 0.00396348  0.00370129  0.00121299 ... -0.000635    0.0009616
  -0.00455817]
 ...
 [ 0.01241955 -0.0013349  -0.00415254 ... -0.00229507 -0.00596432
   0.00516253]
 [ 0.01278539  0.00394052 -0.00266557 ... -0.00124086 -0.00576593
   0.00266049]
 [ 0.01039918  0.00220814 -0.0085334  ...  0.00334216 -0.00515078
   0.00147772]], shape=(100, 65), dtype=float32)


In [0]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 4.5842603e-03  1.3953068e-03  1.0012606e-03 -1.0883755e-03
 -1.6069482e-04  1.4236621e-03  5.4281433e-03  3.9126808e-03
  1.6532218e-03  3.4370690e-03  3.1936629e-04 -3.3490807e-03
 -7.1061042e-04 -2.1285343e-03  1.5749354e-03 -4.4176560e-03
 -6.8169292e-03 -1.4768081e-03 -2.9479014e-03  7.8048470e-04
 -1.6846774e-03 -1.4856020e-03  4.3962407e-03 -4.1605141e-03
 -5.5116685e-03  5.3180265e-05  2.5392587e-03  3.5015070e-03
  1.0304153e-04 -2.6488607e-04  1.0280669e-03  3.3745458e-03
 -5.3748023e-05  5.3398749e-03  1.0469414e-03 -1.1741404e-03
  1.1837165e-03  8.5717533e-05  1.8214770e-03 -3.1604818e-03
  2.1184504e-04  3.7598985e-03 -9.7343754e-03 -9.9674240e-04
  4.1490071e-03  2.8484599e-03 -4.3091923e-03 -8.4307082e-03
 -2.7076167e-04 -9.5313013e-04 -3.6597385e-03  1.5404974e-03
  1.8288286e-03 -3.1366819e-03 -2.1116617e-03 -2.0738395e-03
  3.2301163e-03  1.0542050e-03 -3.8783019e-03 -9.6937083e-04
 -4.2649033e-03 -1.2432381e-03 -2.6670808e-03  4.8370371e-03
 -6.060701

In [0]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"HtGxABtTI&xL&KJfBqesotFtdM3$ Pn b reIYIsRyJZSMvbmd.a:h?e!HeuUAlG;feEDqRJ?m:VDFBU,LAay;,ZxzG'gykelMlW"

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 10s 57ms/step - loss: 2.6277
Epoch 2/50
172/172 [==============================] - 10s 55ms/step - loss: 1.9191
Epoch 3/50
172/172 [==============================] - 9s 55ms/step - loss: 1.6689
Epoch 4/50
172/172 [==============================] - 9s 55ms/step - loss: 1.5343
Epoch 5/50
172/172 [==============================] - 9s 55ms/step - loss: 1.4521
Epoch 6/50
172/172 [==============================] - 9s 55ms/step - loss: 1.3950
Epoch 7/50
172/172 [==============================] - 9s 55ms/step - loss: 1.3518
Epoch 8/50
172/172 [==============================] - 9s 55ms/step - loss: 1.3134
Epoch 9/50
172/172 [==============================] - 9s 55ms/step - loss: 1.2801
Epoch 10/50
172/172 [==============================] - 9s 55ms/step - loss: 1.2495
Epoch 11/50
172/172 [==============================] - 9s 55ms/step - loss: 1.2188
Epoch 12/50
172/172 [==============================] - 9s 55ms/step - loss: 1.1878
Epoch 13/50

In [0]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [0]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeof this woman?

Nurse:
I know him well: you have a stomachlife, when I

PARIS:
Why, how now, son? what's this?

LADY CAPULET:
What think you, if you call?

KING RICHARD II:
Fly to the duke.

LUCIO:
How now? more behind in golden! woe me not all decore.

ROMEO:
What, house-bed laid me not? why, 'tis news I have done this the princh and not true.

TRANIO:
What says he?

CATESBY:
Here, sir; here had she spenst me where the affection give than guest
And by the man that profish'd thus applauant;
For thou dost know our vain tyrant.

PAULINA:
Tut, tult KINGHAM:
Think you, my lord, for I no matter,
Now much off that title it did sweetly, measure his harvest stripes home: but with a mighty soul,
age! I am my neither violent fire,
For thy consent to my dife-pirchangemast, seat!
The senseless virtue o
